In [67]:
import requests
import datetime

def get_current_game_info(team_id):
    games_url = 'http://statsapi.mlb.com/api/v1/schedule/games/?sportId=1'
    result = requests.get(games_url)
    games = result.json()['dates'][0]['games']

    # Get current date in UTC
    current_date_utc = datetime.datetime.utcnow()

    for game in games:
        home_team_id = game['teams']['home']['team']['id']
        away_team_id = game['teams']['away']['team']['id']
        
        game_date = datetime.datetime.strptime(game['gameDate'], "%Y-%m-%dT%H:%M:%SZ")

        if (home_team_id == team_id or away_team_id == team_id) and game_date.date() == current_date_utc.date():
            home_team_name = game['teams']['home']['team']['name']
            away_team_name = game['teams']['away']['team']['name']
            game_pk = game['gamePk']
            return game_pk, home_team_name, away_team_name

    # Return None if no game is found
    return None, None, None

# ID for Philadelphia Phillies
phillies_id = 143

game_id, home_team, away_team = get_current_game_info(phillies_id)

if game_id is not None:
    print(f"Game ID for current Phillies game: {game_id}")
    print(f"Home Team: {home_team}")
    print(f"Away Team: {away_team}")
else:
    print("No game found for the Philadelphia Phillies today.")

Game ID for current Phillies game: 717408
Home Team: Philadelphia Phillies
Away Team: San Diego Padres


In [68]:
import requests

def get_game_info(api_url):
    response = requests.get(api_url)
    json_data = response.json()

    game_info = {}

    # get list of innings from json
    innings = json_data.get('innings', [])

    # iterate over each inning
    for inning in innings:
        inning_num = inning['num']

        # get the home and away runs for each inning
        home_runs = inning['home'].get('runs', 0)
        away_runs = inning['away'].get('runs', 0)

        game_info[inning_num] = {
            'home_runs': home_runs,
            'away_runs': away_runs,
        }

    # General game info
    game_info['general'] = {
        'balls': json_data.get('balls', 0),
        'strikes': json_data.get('strikes', 0),
        'outs': json_data.get('outs', 0),
    }

    # Extract info about who's on base
    game_info['onBase'] = {
        'first': bool(json_data.get('defense', {}).get('first', {})),
        'second': bool(json_data.get('defense', {}).get('second', {})),
        'third': bool(json_data.get('defense', {}).get('third', {})),
    }

    # Total runs, hits, and errors for each team
    game_info['total'] = {
        'home': {
            'runs': json_data.get('teams', {}).get('home', {}).get('runs', 0),
            'hits': json_data.get('teams', {}).get('home', {}).get('hits', 0),
            'errors': json_data.get('teams', {}).get('home', {}).get('errors', 0),
        },
        'away': {
            'runs': json_data.get('teams', {}).get('away', {}).get('runs', 0),
            'hits': json_data.get('teams', {}).get('away', {}).get('hits', 0),
            'errors': json_data.get('teams', {}).get('away', {}).get('errors', 0),
        }
    }

    return game_info

# Example usage:
api_url = 'https://statsapi.mlb.com/api/v1/game/717408/linescore'
game_info = get_game_info(api_url)

game_info

{1: {'home_runs': 0, 'away_runs': 0},
 2: {'home_runs': 0, 'away_runs': 3},
 3: {'home_runs': 0, 'away_runs': 0},
 4: {'home_runs': 0, 'away_runs': 0},
 5: {'home_runs': 0, 'away_runs': 0},
 6: {'home_runs': 2, 'away_runs': 0},
 7: {'home_runs': 1, 'away_runs': 0},
 8: {'home_runs': 3, 'away_runs': 1},
 9: {'home_runs': 0, 'away_runs': 0},
 'general': {'balls': 1, 'strikes': 1, 'outs': 3},
 'onBase': {'first': True, 'second': True, 'third': True},
 'total': {'home': {'runs': 6, 'hits': 10, 'errors': 0},
  'away': {'runs': 4, 'hits': 9, 'errors': 1}}}

# Endpoint documentation

See this page for more info on MLB API endpoints (e.g. how to call linescore)

https://github.com/toddrob99/MLB-StatsAPI/blob/master/statsapi/endpoints.py